# library import

In [1]:
import datetime as dt
from dateutil.relativedelta import relativedelta
import glob
import numpy as np
import pandas as pd
import re
import swifter

# Data Load and create yasa DataFrame

In [2]:
KIND_LABEL = pd.read_csv("../data/test.csv")["kind"].unique()

In [3]:
def create_year_month(date_string):
    date_string = re.sub(r"[月|年]", "", date_string)
    if date_string[:2] == "平成":
        return str(1988 + int(date_string[2:4])) + "-" + date_string[4:].zfill(2)
    elif date_string[:2] == "令和":
        if date_string[2:3] == "元":
            return str(2019) + "-" + date_string[3:].zfill(2)
        else:
            return str(2018 + int(date_string[2:4])) + "-" + date_string[4:].zfill(2)

In [4]:
df_yasai = pd.DataFrame()

for kind in KIND_LABEL:
    for file in glob.glob(f"../raw_data/yasai_data/{kind}/*.csv"):
        df_tmp = pd.read_csv(file)
        df_tmp = df_tmp.iloc[:, :-1].drop(columns=["合計"]).melt(id_vars=["産地", "産地.1"], var_name="year-month")
        df_tmp = df_tmp.query("産地 != '合計'").rename(columns={"産地": "area", "産地.1": "数値"}).replace(["-"], np.nan)
        df_tmp["year-month"] = df_tmp["year-month"].map(create_year_month)
        df_tmp["kind"] = kind
        df_tmp["value"] = df_tmp["value"].map(lambda x: int(x), na_action="ignore")
        
        df_yasai = pd.concat([df_yasai, df_tmp], ignore_index=True)
        
DELETE_AREA = [
    "中国",
    "オランダ",
    "メキシコ",
    "オーストラリア",
    "ニュージーランド",
    "オランダ",
    "その他外国",
    "ベトナム",
    "米国",
    "韓国",
    "カナダ",
    "コスタリカ",
    "ペルー",
    "イタリア",
    "北朝鮮",
    "南アフリカ",
    "トルコ",
    "フィリピン",
    "タイ"
]
df_yasai = df_yasai.query("area not in @DELETE_AREA")

In [5]:
df_train = pd.read_csv("../data/train.csv", parse_dates=["date"])
df_test = pd.read_csv("../data/test.csv", parse_dates=["date"])
df_weather = pd.read_csv("../data/weather_v3.csv")
df_emergency = pd.read_csv("../data/emergencydata.csv", parse_dates=["date"])

In [6]:
def create_merge_df(train, test):
    train["data_flag"] = "train"
    test["data_flag"] = "test"
    return pd.concat([df_train, df_test]).query("not area.str.contains('中国')")

In [7]:
df_merge = create_merge_df(df_train, df_test)

# Preprocessing

## 5移動平均と差分系列

In [8]:
def moving_avg(df, kind):
    df = df.query("kind == @kind").copy()
    df["amount_moving_avg"] = df["amount"].rolling(window=5).mean()
    df["amount_diff"] = df["amount"].diff()
    
    return df

In [9]:
df_merge_2 = pd.DataFrame()
for kind in KIND_LABEL:
    df_merge_2 = pd.concat([df_merge_2, moving_avg(df_merge, kind)])

## area split

In [10]:
def create_df_column_split(df, column, split_str, expand=True, drop_column=False, rename_columns=None):
    if drop_column:
        df_original = df.drop(columns=[column])
    else:
        df_original = df

    df_split = pd.concat(
        [
            df_original,
            df[column].str.split(split_str, expand=expand)
        ],
        axis=1
    )

    if rename_columns is None:
        return df_split
    else:
        return df_split.rename(columns=rename_columns)

In [11]:
df_merge_2 = create_df_column_split(df_merge_2, "area", "_", expand=False, drop_column=True)

## date split

In [12]:
def date_split(df, date_column):
    df["year"] = df[date_column].dt.year
    df["month"] = df[date_column].dt.month
    df["year-month"] = df[date_column].dt.strftime("%Y-%m")
    df["month-range"] = df["month"].astype(str) + "-" + df[date_column].dt.day.map(lambda x: "上旬" if 0 <= x <= 10 else "中旬" if 11 <= x <= 20 else "下旬")
    df["dayofweek"] = df[date_column].dt.dayofweek.map({0: "Mon", 1: "Tue", 2: "Wed", 3: "Thu", 4: "Fri", 5: "Sat", 6: "Sun"})
    
    return df

In [13]:
df_merge_2 = date_split(df_merge_2, "date")

## create kind_detail

In [14]:
KIND_DETAIL_DICT = {
    "きゅうり": {"冬春きゅうり": [12, 1, 2, 3, 4, 5, 6], "夏秋きゅうり": [7, 8, 9, 10, 11]},
    "こまつな": {"こまつな": [i for i in range(1, 13)]},
    "じゃがいも": {"じゃがいも": [i for i in range(1, 13)]},
    "そらまめ": {"そらまめ": [i for i in range(1, 13)]},
    "だいこん": {"秋冬だいこん": [10, 11, 12, 1, 2, 3], "春だいこん": [4, 5, 6], "夏だいこん": [7, 8, 9]},
    "なましいたけ": {"なましいたけ": [i for i in range(1, 13)]},
    "にんじん": {"冬にんじん": [11, 12, 1, 2, 3], "春夏にんじん": [4, 5, 6, 7], "秋にんじん": [8, 9, 10]},
    "ねぎ": {"春ねぎ": [4, 5, 6], "夏ねぎ": [7, 8, 9], "秋冬ねぎ": [10, 11, 12, 1, 2, 3]},
    "はくさい": {"秋冬はくさい": [10, 11, 12, 1, 2, 3], "春はくさい": [4, 5, 6], "夏はくさい": [7, 8, 9]},
    "ほうれんそう": {"ほうれんそう": [i for i in range(1, 13)]},
    "キャベツ": {"冬キャベツ": [11, 12, 1, 2, 3], "春キャベツ": [4, 5, 6], "夏秋キャベツ": [7, 8, 9, 10]},
    "セルリー": {"セルリー": [i for i in range(1, 13)]},
    "トマト": {"冬春トマト": [12, 1, 2, 3, 4, 5, 6], "夏秋トマト": [7, 8, 9, 10, 11]},
    "ミニトマト": {"冬春ミニトマト": [12, 1, 2, 3, 4, 5, 6], "夏秋ミニトマト": [7, 8, 9, 10, 11]},
    "ピーマン": {"夏秋ピーマン": [6, 7, 8, 9, 10], "冬春ピーマン": [11, 12, 1, 2, 3, 4, 5]},
    "レタス": {"冬レタス": [11, 12, 1, 2, 3], "春レタス": [4, 5], "夏秋レタス": [6, 7, 8, 9, 10]}
}

def judge_kind_detail(row):
    for k_kind, v_kind_dict in KIND_DETAIL_DICT.items():
        if row["kind"] == k_kind:
            for k_kind_detail, v_month_list in v_kind_dict.items():
                if row["month"] in v_month_list:
                    return k_kind_detail
                
def create_kind_detail(df):
    df = df.copy()
    df = df[df["kind"].isin(KIND_DETAIL_DICT.keys())]
    df["kind_detail"] = df.apply(judge_kind_detail, axis=1)
    return df

In [15]:
df_merge_2 = create_kind_detail(df_merge_2)

## yasai weight calculation & stack

In [16]:
def create_preprocess_yasai(df):
    df = df[df["year-month"] >= "2015-01"].copy()
    df["month"] = df["year-month"].map(lambda x: x[5:]).astype(int)
    df = df.groupby(["area", "month", "数値", "kind"])["value"].mean().reset_index()
    
    return df


df_yasai_preprocess = create_preprocess_yasai(df_yasai)

In [17]:
def expand_area(row):
    area_only_list = [i for i in row["area"] if i != "各地"]
    df_yasai_tmp = df_yasai_preprocess.query("数値 == '数量'")
    df_yasai_tmp = df_yasai_preprocess[
        (df_yasai_preprocess["kind"] == row["kind"])
        & (df_yasai_preprocess["month"] == row["month"])
        & (~df_yasai_preprocess["area"].isin(area_only_list))
        & (df_yasai_preprocess["value"] > 0)
    ]
    
    if "各地" in row["area"]:
        return area_only_list + df_yasai_tmp["area"].unique().tolist()
    else:
        return row["area"]
    
    
def weight_judge(row):
    weight_dict = {}
    df_yasai_tmp = df_yasai_preprocess.query("数値 == '数量'")
    for idx, area in enumerate(row["area"]):
        try:
            weight_dict[f"area_{idx}"] = df_yasai_tmp[
                   (df_yasai_tmp["kind"] == row["kind"])
                   & (df_yasai_tmp["area"] == area) 
                   & (df_yasai_tmp["month"] == row["month"])
                ]["value"].iloc[0]
        except IndexError:
            print(row["area"], row["kind"], row["year-month"])
    
    return weight_dict


def weight_calcurate(df):
    df = df.reset_index(drop=True)
    df["area"] = df.swifter.apply(expand_area, axis=1)
    df["amount_weight"] = df.swifter.apply(weight_judge, axis=1)
    df = pd.concat([df, df["amount_weight"].apply(pd.Series)], axis=1)
    
    EXPAND_RANGE = df["area"].map(lambda x: len(x)).max()
    df["total_amount"] = df[[f"area_{i}" for i in range(EXPAND_RANGE)]].sum(skipna=True, axis=1)
    for i in range(EXPAND_RANGE):
        df[f"area_{i}"] = (df[f"area_{i}"] / df["total_amount"]) * df["amount"]
    df = pd.concat([df, df["area"].apply(pd.Series)], axis=1)
    df = df.drop(columns=["area", "amount_weight", "total_amount"])
        
    df_update = pd.DataFrame()
    delete_list = [i for i in range(EXPAND_RANGE)] + [f"area_{i}" for i in range(EXPAND_RANGE)]
    for i in range(EXPAND_RANGE):
        df_update = pd.concat(
            [
                df_update,
                df[[i for i in df.columns if i not in delete_list] + [i, f"area_{i}"]]\
                        .rename(columns={i: "area", f"area_{i}": "weight_amount"}).query("area == area")
            ],
            ignore_index=True
       )
    return df_update

In [18]:
df_merge_2 = weight_calcurate(df_merge_2[df_merge_2["year-month"] >= "2015-01"])

Pandas Apply:   0%|          | 0/27742 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/27742 [00:00<?, ?it/s]

## avg_price merge

In [19]:
df_merge_2 = df_merge_2.merge(
    df_yasai_preprocess.query("数値 == '金額'").drop(columns=["数値"]),
    how="left",
    on=["kind", "month", "area"]
).rename(columns={"value": "total_month_price"})\
    .merge(
    df_yasai_preprocess.query("数値 == '平均価格'").drop(columns=["数値"]),
    how="left",
    on=["kind", "month", "area"]
).rename(columns={"value": "avg_month_price"})

## weather merge

In [20]:
df_earthquake = pd.read_csv("../data/earthquakeData_v2.csv", parse_dates=["date"]).rename(columns={"震源(都道府県)": "area"})
df_earthquake["year-month"] = df_earthquake["date"].dt.strftime("%Y-%m")
df_earthquake.groupby(["year-month", "area"]).size().reset_index().rename(columns={0: "earthquake_count"})


def typhoon_fillna(df, agg):
    df = df.copy()
    WEATHER_AREA_DICT = {
        "仙台": "宮城",
        "佐賀": "佐賀",
        "前橋": "群馬",
        "千葉": "千葉",
        "名古屋": "愛知",
        "和歌山": "和歌山",
        "宇都宮": "栃木",
        "宮崎": "宮崎",
        "山形": "山形",
        "帯広": "北海道",
        "徳島": "徳島",
        "新潟": "新潟",
        "東京": "東京",
        "松山": "愛媛",
        "横浜": "神奈川",
        "水戸": "茨城",
        "浜松": "静岡",
        "熊本": "熊本",
        "熊谷": "埼玉",
        "甲府": "山梨",
        "盛岡": "岩手",
        "神戸": "兵庫",
        "福岡": "福岡",
        "福島": "福島",
        "秋田": "秋田",
        "那覇": "沖縄",
        "長崎": "長崎",
        "長野": "長野",
        "青森": "青森",
        "高松": "香川",
        "高知": "高知",
        "鹿児島": "鹿児島"
    }
    df["area"] = df["area"].map(WEATHER_AREA_DICT)
    df["typhoon_approach"] = df["typhoon_approach"].fillna(0)
    
    df = df.merge(
        df_earthquake.groupby(["year-month", "area"]).size().reset_index().rename(columns={0: "earthquake_count"}),
        how="left",
        on=["year-month", "area"]
    )
    df["earthquake_count"] = df["earthquake_count"].fillna(0)
    return df

In [21]:
df_weather = typhoon_fillna(df_weather, "mean")

In [22]:
def weather_area(area):
    if area == "大分":
        return "熊本"
    elif area == "岐阜":
        return "愛知"
    else:
        return area


def lag_weather_add(df, n_shift):
    df_weather_tmp = df_weather.drop(columns=["region"])
    df_weather_tmp.columns = [f"{i}_{n_shift}prev" if i not in ["year-month", "month", "area"] else i for i in df_weather_tmp.columns]
    df_weather_tmp["merge-year-month"] = df_weather_tmp["year-month"]
    
    df["area"] = df["area"].map(weather_area)
    df["merge-year-month"] = pd.to_datetime(df["year-month"]).dt.date + relativedelta(months=-n_shift)
    df["merge-year-month"] = df["merge-year-month"].map(lambda x: x.strftime("%Y-%m"))
    return df.merge(df_weather_tmp.drop(columns="year-month"), how="inner", on=["merge-year-month", "area"]).drop(columns=["merge-year-month"])

In [23]:
for i in range(1, 13):
    df_merge_2 = lag_weather_add(df_merge_2, i)

## emergency merge

In [24]:
df_merge_2 = df_merge_2.merge(df_emergency, how="left", on="date")
df_merge_2["emagency"] = df_merge_2["emagency"].fillna(0)

In [25]:
df_merge_2.to_csv("../data/merge_data_preprocessing_v7.csv", index=False)